In [1]:
import numpy as np
import pandas as pd

import en_nlp_utils

In [2]:
# Define default source path
SRC_PATH = "src/"

In [3]:
# Load dataset
df_agoda_hotel = pd.read_csv(SRC_PATH + "agoda_hotels_details_valid.csv")
df_agoda_review = pd.read_csv(SRC_PATH + "agoda_reviews_details_cleaned.csv")

df_booking_hotel = pd.read_csv(SRC_PATH + "booking_hotels_details_valid.csv")
df_booking_review = pd.read_csv(SRC_PATH + "booking_reviews_details_cleaned.csv")

In [4]:
en_nlp_utils.check_null(df_agoda_hotel, "df_agoda_hotel")
en_nlp_utils.check_null(df_booking_hotel, "df_booking_hotel")

df_agoda_hotel:
hotel_id                             0
hotel_name                           0
hotel_url                            0
hotel_address                        0
total_reviews                        0
overall_rating                       0
overall_rating_category              0
rating_location                      0
rating_service                       0
rating_cleanliness                   0
rating_room_comfort_and_quality    168
rating_value_for_money               0
rating_facilities                    0
dtype: int64
Total rows in df_agoda_hotel: 361 

df_booking_hotel:
url                         0
hotel_name                  0
address                     0
stars                      12
reviews_count               0
reviews_grade               0
grades_staff                0
grades_facilities           0
grades_cleanliness          0
grades_comfort              0
grades_value_for_money      0
grades_location             0
sustainable_level          57
most_popular_facilit

## 1) Hotel Data

### a) Processing df_agoda_hotel

In [5]:
# Remove unnecessary column
df_agoda_hotel.drop(columns=["hotel_url"], inplace=True)

In [6]:
# Create "source" column
df_agoda_hotel["source"] = "Agoda"

In [7]:
# Fill in NULL values of "rating_room_comfort_and_quality" column using its average rating
mean_rating = df_agoda_hotel["rating_room_comfort_and_quality"].mean()
mean_rating = round(mean_rating, 1)

df_agoda_hotel["rating_room_comfort_and_quality"].fillna(mean_rating, inplace=True)

In [8]:
# Convert "total_reviews" to integer format
df_agoda_hotel["total_reviews"] = df_agoda_hotel["total_reviews"].astype(int)

### b) Processing df_booking_hotel

In [9]:
# Remove unnecessary columns
df_booking_hotel.drop(columns=["url", "stars", "sustainable_level", "most_popular_facilities", "highlights"], inplace=True)

In [10]:
# Rename columns for consistency
df_booking_hotel = df_booking_hotel.rename(columns={"address": "hotel_address",
                                                   "reviews_count": "total_reviews",
                                                   "reviews_grade": "overall_rating",
                                                   "grades_location": "rating_location",
                                                   "grades_staff": "rating_service",
                                                   "grades_cleanliness": "rating_cleanliness",
                                                   "grades_comfort": "rating_room_comfort_and_quality",
                                                   "grades_value_for_money": "rating_value_for_money",
                                                   "grades_facilities": "rating_facilities"})

In [11]:
# List key columns that are used to identify the duplicate rows
key_columns_hotel = ["hotel_name", "hotel_address"]

# Find out duplicated rows and keep last one
duplicated_rows_hotel = df_booking_hotel[df_booking_hotel.duplicated(subset=key_columns_hotel, keep="last")]

# Remove duplicated rows and keep last one
df_booking_hotel = df_booking_hotel.drop_duplicates(subset=key_columns_hotel, keep="last").reset_index(drop=True)

In [12]:
# Create "hotel_id" column
df_booking_hotel["hotel_id"] = range(1, len(df_booking_hotel) + 1)

# Create "source" column
df_booking_hotel["source"] = "Booking"

In [13]:
# Create "overall_rating_category" column

# Note:
# overall_rating_category = "Exceptional": overall_rating >= 9
# overall_rating_category = "Excellent": overall_rating >= 8 and < 9
# overall_rating_category = "Very Good": overall_rating >= 7 and < 8
# overall_rating_category = "Good": overall_rating >= 6 and < 7
# overall_rating_category = "Below Expectation": overall_rating < 6

df_booking_hotel.loc[df_booking_hotel["overall_rating"] >= 9, "overall_rating_category"] = "Exceptional"
df_booking_hotel.loc[(df_booking_hotel["overall_rating"] >= 8) & (df_booking_hotel["overall_rating"] < 9), "overall_rating_category"] = "Excellent"
df_booking_hotel.loc[(df_booking_hotel["overall_rating"] >= 7) & (df_booking_hotel["overall_rating"] < 8), "overall_rating_category"] = "Very Good"
df_booking_hotel.loc[(df_booking_hotel["overall_rating"] >= 6) & (df_booking_hotel["overall_rating"] < 7), "overall_rating_category"] = "Good"
df_booking_hotel.loc[df_booking_hotel["overall_rating"] < 6, "overall_rating_category"] = "Below Expectation"

### c) Merge datasets

In [14]:
# Merge datasets
df_merged_hotel = pd.concat([df_agoda_hotel, df_booking_hotel], ignore_index=True)

### d) Processing merged data

#### Extract/Fill in postal codes¶

In [15]:
# Extract postal code from "hotel_address" column and add as new column "hotel_postal_code"
df_merged_hotel["hotel_postal_code"] = df_merged_hotel["hotel_address"].str.extract(r"\b(\d{6})\b")
df_merged_hotel.head(2)

,hotel_id,hotel_name,hotel_address,total_reviews,overall_rating,overall_rating_category,rating_location,rating_service,rating_cleanliness,rating_room_comfort_and_quality,rating_value_for_money,rating_facilities,source,hotel_postal_code
0,408551,Dorsett Singapore,"333 New Bridge Road, Chinatown, Singapore, Sin...",10337,8.2,Excellent,9.0,8.4,8.3,8.1,8.1,7.9,Agoda,088765
1,1635,M Hotel Singapore,"81 Anson Road, CBD, Singapore, Singapore, 079908",9034,8.1,Excellent,8.2,8.1,8.5,8.2,8.0,8.0,Agoda,079908


In [16]:
# Find the rows of hotel without postal codes 
rows_missing_postal_codes = df_merged_hotel[df_merged_hotel["hotel_postal_code"].isnull()]
rows_missing_postal_codes[["hotel_name", "hotel_address", "hotel_postal_code", "source"]]

,hotel_name,hotel_address,hotel_postal_code,source
92,"PARKROYAL COLLECTION Marina Bay, Singapore","6 Raffles Boulevard Marina Square, Marina Bay,...",NaN,Agoda
102,Hotel 81 Premier Star,"31 Lorong 18 Geylang, Geylang, Singapore, Sing...",NaN,Agoda
104,Holiday Inn Express Singapore Clarke Quay,"2 Magazine Road, Clarke Quay, Singapore, Singa...",NaN,Agoda
135,The Sultan,"101 Jalan Sultan, Bugis, Singapore, Singapore",NaN,Agoda
140,30 Bencoolen,"30 Bencoolen Street, Bugis, Singapore, Singapo...",NaN,Agoda
249,Lion Peak Hotel Bugis,"103 Beach Road, Bugis, Singapore, Singapore",NaN,Agoda
280,Citadines Raffles Place Singapore,"88 Market Street, CBD, Singapore, Singapore, 4...",NaN,Agoda
295,The Westin Singapore,"12 Marina View, Asia Square Tower 2, CBD, Sing...",NaN,Agoda
306,JXIN HOTEL 嘉鑫酒店,"Lorong 10 Geylang, 9, Geylang, Singapore, Sing...",NaN,Agoda
326,Fragrance Hotel - Oasis,"435 Balestier Road, Novena, Singapore, Singapore",NaN,Agoda


In [17]:
# Manually find out the postal codes
replacement_postal_codes = ["039594", "398828", "059573", "199002", "189621", "189704", "048948", 
                            "018961", "399042", "329816", "519121", "039797", "828670", "078972", 
                            "089970", "099593", "398961"]

# Replace the missing postal codes with values in "replacement_postal_codes"
for i, postal_code in enumerate(replacement_postal_codes):
    df_merged_hotel.loc[rows_missing_postal_codes.index[i], "hotel_postal_code"] = postal_code

#### Map postal codes to longitude and latitude¶

In [18]:
# Load postal data
df_postal = pd.read_csv(SRC_PATH + "SG_postal.csv")

# Change "postal_code" data type as str
df_postal["postal_code"] = df_postal["postal_code"].astype(str)

# Add a "0" in front of postal_code of df_postal with 5 digits
df_postal["postal_code"] = df_postal["postal_code"].apply(lambda x: x.zfill(6))

# Merge df_merged_hotel and df_postal based on postal codes
df_merged_hotel = pd.merge(df_merged_hotel, df_postal, how="left", left_on="hotel_postal_code", right_on="postal_code")

In [19]:
# Find the rows of hotel without latitude/longitude
rows_missing_lat_lon = df_merged_hotel[df_merged_hotel["lat"].isnull()]
rows_missing_lat_lon[["hotel_name", "hotel_postal_code", "source"]]

,hotel_name,hotel_postal_code,source
84,Mondrian Singapore Duxton,089970,Agoda
111,Hotel 81 Kovan,543636,Agoda
141,lyf one-north Singapore,139409,Agoda
230,Aqueen Prestige Hotel Lavender,207577,Agoda
334,Pullman Singapore Hill Street,179949,Agoda
335,Galaxy Pods @ Chinatown,059505,Agoda
430,lyf one-north Singapore,139409,Booking
454,Mondrian Singapore Duxton,089970,Booking
502,Hotel Faber Park Singapore - Handwritten Colle...,099593,Booking
550,Pullman Singapore Hill Street,179949,Booking


In [20]:
# Manually find out the latitude/longitude
replacement_lat_lon = ["1.2789417774465628, 103.84267813216216", "1.3544863670909781, 103.87891156633059",
                       "1.3010513983161585, 103.78810722400226", "1.3132099146320708, 103.86075809284718",
                       "1.2939472946122026, 103.85066931050956", "1.2837052738743098, 103.84557825283808",
                       "1.3010513983161585, 103.78810722400226", "1.2789417774465628, 103.84267813216216",
                       "1.2703444013142564, 103.81339682400237", "1.2939472946122026, 103.85066931050956",
                       "1.2915950992469878, 103.83533876633084", "1.2837052738743098, 103.84557825283808"]

# Replace the missing latitude/longitude with values in "replacement_lat_lon"
for i, lat_lon in enumerate(replacement_lat_lon):
    # Split the latitude and longitude values
    lat, lon = map(float, lat_lon.split(', '))
    
    # Round the latitude and longitude to four decimal places
    lat = round(lat, 4)
    lon = round(lon, 4)
    
    # Update latitude/longitude values into df_merged_hotel
    df_merged_hotel.loc[rows_missing_lat_lon.index[i], "lat"] = lat
    df_merged_hotel.loc[rows_missing_lat_lon.index[i], "lon"] = lon

#### Remove unnecessary columns and reorder columns

In [21]:
# Remove unnecessary columns
df_merged_hotel.drop(columns=["postal_code", "street_name"], inplace=True)

In [22]:
# New column order
new_column_order = ["source", "hotel_id", "hotel_name", "hotel_address", "hotel_postal_code", "lat", "lon",
                    "total_reviews", "overall_rating", "overall_rating_category", "rating_location",
                    "rating_service", "rating_cleanliness", "rating_room_comfort_and_quality",
                    "rating_value_for_money", "rating_facilities"]

# Reorder columns
df_merged_hotel = df_merged_hotel.reindex(columns=new_column_order)

In [23]:
# Check NULL values
en_nlp_utils.check_null(df_merged_hotel, "df_merged_hotel")

df_merged_hotel:
source                             0
hotel_id                           0
hotel_name                         0
hotel_address                      0
hotel_postal_code                  0
lat                                0
lon                                0
total_reviews                      0
overall_rating                     0
overall_rating_category            0
rating_location                    0
rating_service                     0
rating_cleanliness                 0
rating_room_comfort_and_quality    0
rating_value_for_money             0
rating_facilities                  0
dtype: int64
Total rows in df_merged_hotel: 584 



## 2) Review Data

### a) Processing df_agoda_review

In [24]:
# Remove unnecessary column
df_agoda_review.drop(columns=["rating_cat", "reviewer", "stay_details"], inplace=True)

In [25]:
# Rename columns for consistency
df_agoda_review = df_agoda_review.rename(columns={"review_score_cat": "review_score_category"})

In [26]:
# Create "source" column
df_agoda_review["source"] = "Agoda"

In [27]:
# Create "sentiment" column

# Note:
# positive: review_score >= 6
# negative: review_score < 6

df_agoda_review.loc[df_agoda_review["review_score"] >= 6, "sentiment"] = "positive"
df_agoda_review.loc[df_agoda_review["review_score"] < 6, "sentiment"] = "negative"

In [28]:
# Fill in NULL values of "group_name" and "room_type" columns using "Other"
df_agoda_review["group_name"].fillna("Other", inplace=True)
df_agoda_review["room_type"].fillna("Other", inplace=True)

In [29]:
# Check NULL values
en_nlp_utils.check_null(df_agoda_review, "df_agoda_review")

df_agoda_review:
hotel_id                  0
hotel_name                0
country                   0
group_name                0
room_type                 0
stay_length               0
stay_date                 0
review_score              0
review_score_category     0
review_date               0
review_title              0
review                    0
review_cleaned_v1        97
source                    0
sentiment                 0
dtype: int64
Total rows in df_agoda_review: 31582 



In [30]:
df_agoda_review.head(1)

,hotel_id,hotel_name,country,group_name,room_type,stay_length,stay_date,review_score,review_score_category,review_date,review_title,review,review_cleaned_v1,source,sentiment
0,1007,"PARKROYAL on Kitchener Road, Singapore",India,Couple,Superior Corner Room with 1 King Bed,6.0,01/01/2024,9.6,Exceptional,28/01/2024,Gr8,Awesome in all aspects,awesome aspect,Agoda,positive


### b) Processing df_booking_review

#### Remove duplicated rows

In [31]:
# Convert "stay_date" and "review_date" columns as datetime data type
df_booking_review["stay_date"] = pd.to_datetime(df_booking_review["stay_date"], format="%Y-%m-%d")
df_booking_review["review_date"] = pd.to_datetime(df_booking_review["review_date"], format="%d/%m/%Y")

# Convert "stay_date" and "review_date" columns as "dd/mm/yyyy" format
df_booking_review["stay_date"] = df_booking_review["stay_date"].dt.strftime("%d/%m/%Y")
df_booking_review["review_date"] = df_booking_review["review_date"].dt.strftime("%d/%m/%Y")

In [32]:
# List key columns that are used to identify the duplicate rows
key_columns_review = ["hotel_name", "user_name", "country", "rating", "review_date", "review_title", "happy_review", "unhappy_review"]

# Find out duplicated rows and keep first one
duplicated_rows_review = df_booking_review[df_booking_review.duplicated(subset=key_columns_review, keep="first")]

# Remove duplicated rows and keep first one
df_booking_review = df_booking_review.drop_duplicates(subset=key_columns_review, keep="first").reset_index(drop=True)

#### Split happy and unhapply reviews into separate rows

In [33]:
# Create dataframe for happy reviews
df_happy_review = df_booking_review[["hotel_name", "country", "room_info",
                                     "no_of_nights_stayed", "stay_date", "traveller_type",
                                     "review_title", "rating", "happy_review",
                                     "review_date", "happy_review_cleaned_v1"]].reset_index(drop=True)

df_happy_review["sentiment"] = "positive"

# Create dataframe for unhappy reviews
df_unhappy_review = df_booking_review[["hotel_name", "country", "room_info",
                                       "no_of_nights_stayed", "stay_date", "traveller_type",
                                       "review_title", "rating", "unhappy_review",
                                       "review_date", "unhappy_review_cleaned_v1"]].reset_index(drop=True)

df_unhappy_review["sentiment"] = "negative"


# Rename columns for consistency
df_happy_review = df_happy_review.rename(columns={"happy_review": "review",
                                                  "happy_review_cleaned_v1": "review_cleaned_v1"})

df_unhappy_review = df_unhappy_review.rename(columns={"unhappy_review": "review",
                                                      "unhappy_review_cleaned_v1": "review_cleaned_v1"})


# Merge happy and unhappy reviews dataframe
df_booking_review_merged = pd.concat([df_happy_review, df_unhappy_review], ignore_index=True)

In [34]:
# Remove rows when "review" column is NULL
df_booking_review_merged = df_booking_review_merged.dropna(subset=["review"]).reset_index(drop=True)

#### Get "hotel_id" column from df_booking_hotel and insert into df_booking_review_merged

In [35]:
# Get "hotel_id" and "hotel_name" columns from df_booking_hotel
hotel_id_mapping = df_booking_hotel[["hotel_id", "hotel_name"]]

# Merge "hotel_id" column into df_booking_review_merged based on "hotel_name" column
df_booking_review_merged = pd.merge(df_booking_review_merged, hotel_id_mapping, on="hotel_name", how="left")

#### Align to df_agoda_review

In [36]:
# Rename columns for consistency
df_booking_review_merged = df_booking_review_merged.rename(columns={"traveller_type": "group_name", 
                                                                    "room_info": "room_type", 
                                                                    "no_of_nights_stayed": "stay_length", 
                                                                    "rating": "review_score"})

In [37]:
# Fill in NULL values of "country" and "room_type" columns using "Other"
df_booking_review_merged["country"].fillna("Other", inplace=True)
df_booking_review_merged["room_type"].fillna("Other", inplace=True)

# Fill in NULL values of "stay_length" and "hotel_id" columns using "0"
df_booking_review_merged["stay_length"].fillna(0, inplace=True)
df_booking_review_merged["hotel_id"].fillna(0, inplace=True)

In [38]:
# Convert "hotel_id" column to integer format
df_booking_review_merged["hotel_id"] = df_booking_review_merged["hotel_id"].astype(int)

In [39]:
# Create "source" column
df_booking_review_merged["source"] = "Booking"

In [40]:
# Create "review_score_category" column

# Note:
# review_score_category = "Exceptional": review_score >= 9
# review_score_category = "Excellent": review_score >= 8 and < 9
# review_score_category = "Very Good": review_score >= 7 and < 8
# review_score_category = "Good": review_score >= 6 and < 7
# review_score_category = "Below Expectation": review_score < 6

df_booking_review_merged.loc[df_booking_review_merged["review_score"] >= 9, "review_score_category"] = "Exceptional"
df_booking_review_merged.loc[(df_booking_review_merged["review_score"] >= 8) & (df_booking_review_merged["review_score"] < 9), "review_score_category"] = "Excellent"
df_booking_review_merged.loc[(df_booking_review_merged["review_score"] >= 7) & (df_booking_review_merged["review_score"] < 8), "review_score_category"] = "Very Good"
df_booking_review_merged.loc[(df_booking_review_merged["review_score"] >= 6) & (df_booking_review_merged["review_score"] < 7), "review_score_category"] = "Good"
df_booking_review_merged.loc[df_booking_review_merged["review_score"] < 6, "review_score_category"] = "Below Expectation"

In [41]:
# Check NULL values
en_nlp_utils.check_null(df_booking_review_merged, "df_booking_review_merged")

df_booking_review_merged:
hotel_name                0
country                   0
room_type                 0
stay_length               0
stay_date                 0
group_name                0
review_title              1
review_score              0
review                    0
review_date               0
review_cleaned_v1        46
sentiment                 0
hotel_id                  0
source                    0
review_score_category     0
dtype: int64
Total rows in df_booking_review_merged: 8392 



In [42]:
df_booking_review_merged.head(1)

,hotel_name,country,room_type,stay_length,stay_date,group_name,review_title,review_score,review,review_date,review_cleaned_v1,sentiment,hotel_id,source,review_score_category
0,Ascott Raffles Place Singapore,Portugal,Cutler Suite,1.0,01/12/2022,Couple,Exceptional,10.0,"room very spacious and clean, great location",12/12/2022,room spacious clean great location,positive,91,Booking,Exceptional


### c) Merge datasets

In [43]:
# Merge datasets
df_merged_review = pd.concat([df_agoda_review, df_booking_review_merged], ignore_index=True)

In [44]:
# New column order
new_column_order_review = ["source", "hotel_id", "hotel_name", "country", "group_name", 
                           "room_type", "stay_length", "stay_date", "review_score", "review_score_category", 
                           "sentiment", "review_date", "review_title", "review", "review_cleaned_v1" ]

# Reorder columns
df_merged_review = df_merged_review.reindex(columns=new_column_order_review)

In [45]:
# Check NULL values
en_nlp_utils.check_null(df_merged_review, "df_merged_review")

df_merged_review:
source                     0
hotel_id                   0
hotel_name                 0
country                    0
group_name                 0
room_type                  0
stay_length                0
stay_date                  0
review_score               0
review_score_category      0
sentiment                  0
review_date                0
review_title               1
review                     0
review_cleaned_v1        143
dtype: int64
Total rows in df_merged_review: 39974 



#### Standardize the types of "group_name" column

In [46]:
df_merged_review["group_name"].unique()

array(['Couple', 'Family with young children', 'Solo traveler',
       'Business traveler', 'Group', 'Family with teens', 'Other',
       'Family', 'Solo traveller', ' Group', ' Family', ' Couple',
       ' Solo traveller'], dtype=object)

In [47]:
# Remove Spaces before and after the string
df_merged_review["group_name"] = df_merged_review["group_name"].str.strip()

In [48]:
# Change similar values in "group_name" columns
# Change "Family with teens" as "Family with young children"
df_merged_review["group_name"] = df_merged_review["group_name"].replace("Family with teens", "Family with young children")

# Change "Solo traveller" as "Solo traveler"
df_merged_review["group_name"] = df_merged_review["group_name"].replace("Solo traveller", "Solo traveler")

#### Standardize the types of "stay_length" column

In [49]:
# Convert "stay_length" column to integer format
df_merged_review["stay_length"] = df_merged_review["stay_length"].astype(int)

#### Standardize the types of "country" column

In [50]:
# Remove Spaces before and after the string
df_merged_review["country"] = df_merged_review["country"].str.strip()

#### Standardize the types of "room_type" column

In [51]:
# Define room category keywords list
room_categories = {
    "Other": ["cabin", "budget", "day use", "hour stay"],
    "Hostel Room": ["dormitory", "bunk", "capsule", "pod"],
    "Special Room": ["club", "marina", "garden", "city", "fairmont"],
    "Family Suite": ["family"],
    "Deluxe Suite": ["suite", "deluxe", "executive", "junior", "prestige", "signature"],
    "Double Room": ["double", "queen", "twin", "superior"],
    "Single Room": ["single", "private"]
}

# Create a function to match room category keywords to each value in the "room_type" column
def match_room_type(room_type):
    for category, keywords in room_categories.items():
        for keyword in keywords:
            if keyword in room_type.lower():
                return category
    return "Other"

# Add a new column "room_type_new"
df_merged_review["room_type_new"] = df_merged_review["room_type"].apply(match_room_type)

# Replaces the value of the "room_type" column with the value of the "room_type_new" column
df_merged_review["room_type"] = df_merged_review["room_type_new"]

# Remove "room_type_new" column
df_merged_review.drop("room_type_new", axis=1, inplace=True)

In [52]:
# Check NULL values
en_nlp_utils.check_null(df_merged_review, "df_merged_review")

df_merged_review:
source                     0
hotel_id                   0
hotel_name                 0
country                    0
group_name                 0
room_type                  0
stay_length                0
stay_date                  0
review_score               0
review_score_category      0
sentiment                  0
review_date                0
review_title               1
review                     0
review_cleaned_v1        143
dtype: int64
Total rows in df_merged_review: 39974 



In [53]:
df_merged_review.head(1)

,source,hotel_id,hotel_name,country,group_name,room_type,stay_length,stay_date,review_score,review_score_category,sentiment,review_date,review_title,review,review_cleaned_v1
0,Agoda,1007,"PARKROYAL on Kitchener Road, Singapore",India,Couple,Double Room,6,01/01/2024,9.6,Exceptional,positive,28/01/2024,Gr8,Awesome in all aspects,awesome aspect


## 3) Save latest dataset as csv file

In [54]:
# Save latest dataset as csv
df_merged_hotel.to_csv(SRC_PATH + "en_hotel.csv", index=False, encoding="utf-8")
df_merged_review.to_csv(SRC_PATH + "en_hotel_review.csv", index=False, encoding="utf-8")